In [2]:
!pip install peft >> /dev/null

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    default_data_collator,  # understand what this does? 
    get_linear_schedule_with_warmup  # understand what this does?
)
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    PrefixTuningConfig,
    TaskType
)

from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

device = "cuda"
model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
label_column = "text_label"

max_length = 128
lr = 1e-2

num_epochs = 5
batch_size = 8

In [2]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank", "sentences_allagree",)
dataset = dataset["train"].train_test_split(test_size=0.1)

Found cached dataset financial_phrasebank (/home/kamal/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 2037
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 227
    })
})

In [4]:
dataset["validation"] = dataset["test"]

del dataset["test"]

In [5]:
classes = dataset["train"].features["label"].names

dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'The firm is headquartered in Vantaa , southern Finland and has 16 employees .',
 'label': 1,
 'text_label': 'neutral'}

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [8]:
tokenizer.model_max_length

512

In [9]:
def preprocess_function(examples):
    inputs = examples[text_column] # sentence column
    targets = examples[label_column] # text_label column
    
    model_inputs = tokenizer(inputs,
                             max_length=max_length,
                             padding="max_length",
                             truncation=True,
                             return_tensors="pt")
    
    labels = tokenizer(targets,
                       max_length=2,
                       padding="max_length",
                       truncation=True,
                       return_tensors="pt")
    
    labels = labels["input_ids"]
    
    labels[labels == tokenizer.pad_token_id] = -100
    
    model_inputs["labels"] = labels
    
    return model_inputs

In [11]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [12]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)

In [16]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM,
                                 inference_mode=False,
                                 num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.13308583065659835


In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [18]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [00:01<00:00, 16.10it/s]


epoch=0: train_ppl=tensor(4.9247, device='cuda:0') train_epoch_loss=tensor(1.5943, device='cuda:0') eval_ppl=tensor(1.0845, device='cuda:0') eval_epoch_loss=tensor(0.0812, device='cuda:0')


100%|██████████| 29/29 [00:01<00:00, 16.09it/s]


epoch=1: train_ppl=tensor(1.1173, device='cuda:0') train_epoch_loss=tensor(0.1109, device='cuda:0') eval_ppl=tensor(1.0439, device='cuda:0') eval_epoch_loss=tensor(0.0430, device='cuda:0')


100%|██████████| 29/29 [00:01<00:00, 16.13it/s]


epoch=2: train_ppl=tensor(1.0805, device='cuda:0') train_epoch_loss=tensor(0.0774, device='cuda:0') eval_ppl=tensor(1.0363, device='cuda:0') eval_epoch_loss=tensor(0.0357, device='cuda:0')


100%|██████████| 29/29 [00:01<00:00, 16.12it/s]


epoch=3: train_ppl=tensor(1.0732, device='cuda:0') train_epoch_loss=tensor(0.0707, device='cuda:0') eval_ppl=tensor(1.0350, device='cuda:0') eval_epoch_loss=tensor(0.0344, device='cuda:0')


100%|██████████| 29/29 [00:01<00:00, 16.11it/s]

epoch=4: train_ppl=tensor(1.0594, device='cuda:0') train_epoch_loss=tensor(0.0577, device='cuda:0') eval_ppl=tensor(1.0328, device='cuda:0') eval_epoch_loss=tensor(0.0323, device='cuda:0')


In [19]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100

In [20]:
peft_model_path = "/home/kamal/training_files/t5-large_PREFIX_TUNING_SEQ2SEQ"

model.save_pretrained(peft_model_path)

In [21]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained(peft_model_path)

In [22]:
config

PeftConfig(peft_type='PREFIX_TUNING', auto_mapping=None, base_model_name_or_path='t5-large', revision=None, task_type='SEQ_2_SEQ_LM', inference_mode=True)

In [24]:
model_new = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model_new = PeftModel.from_pretrained(model,
                                  peft_model_path)

In [25]:
inputs = tokenizer(
    "The Lithuanian beer market made up 14.41 million liters in January , a rise of 0.8 percent from the year-earlier figure , the Lithuanian Brewers ' Association reporting citing the results from its members .",
    return_tensors="pt",
)

In [26]:
model_new.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model_new.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']
